In [2]:
# 기본 패키지
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import datetime as dt

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
## 이미지에서 한글이 나오게 하는 설정
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%matplotlib inline

## 야후 파이낸스 패키지
import yfinance as yf

## Data Load

In [24]:
tickers = ["QQQ", "SCHD"]

# 여러 티커의 데이터 다운로드
dataset = yf.download(tickers, ignore_tz = True, auto_adjust=True)
dataset = dataset['Close']

[*********************100%%**********************]  2 of 2 completed


## Visualization

### Trend

In [26]:
# dataset.plot()

In [27]:
## 결측 구간 제거
dataset = dataset.dropna()

In [ ]:
# dataset.plot()

In [ ]:
# 스케일 조정
dataset_scaled = (dataset/dataset.iloc[0])*100

In [ ]:
# dataset_scaled.plot()

## Volatility

In [34]:
rets = dataset_scaled.pct_change()*100

In [37]:
## sns.histplot(rets, kde=True)

In [ ]:
def calculate_drawdown(data):
    peak = data.expanding(min_periods=1).max()
    drawdown = (data - peak) / peak
    return drawdown

In [40]:
peak = dataset_scaled.expanding(min_periods=1).max()

In [44]:
## peak['SCHD'].plot()
## dataset_scaled['SCHD'].plot()

In [47]:
drawdown = ((dataset_scaled - peak) / peak)*100

In [55]:
# drawdown.plot()

In [53]:
drawdown.min()

Ticker
QQQ    -35.118733
SCHD   -33.367071
dtype: float64

In [60]:
def calculate_ttr_series(drawdown_series):
    ttr_data = []
    ttr_index = []
    in_drawdown = False
    start_date = None

    for date, value in drawdown_series.items():
        if value < 0 and not in_drawdown:
            # Start of a drawdown
            in_drawdown = True
            start_date = date
        elif value == 0 and in_drawdown:
            # Recovery point
            in_drawdown = False
            ttr_data.append((date - start_date).days)  # TTR in days
            ttr_index.append(date)

    return pd.Series(ttr_data, index=ttr_index)

In [63]:
ttr_dict = {}
for asset in drawdown.columns:
    # Calculate TTR
    ttr_dict[asset] = calculate_ttr_series(drawdown[asset])
ttr = pd.DataFrame(ttr_dict).interpolate(method='time')   

In [66]:
## ttr.plot()

In [67]:
def split_time_series(df, n_days):
    split_days = n_days
    dfs = []
    for i in range(len(df)-split_days+1):
        dfs.append(df[i:split_days+i])
    return dfs

In [68]:
data = dataset_scaled
data = data.resample('M').last()
dfs = split_time_series(data, 12*5)

In [73]:
## calculate Rolling Returns
_ls = []
_idx = []
for _df in dfs:
    __ls = []
    for col in _df.columns:
        total_err = (_df[col][-1]-_df[col][0])/_df[col][0]
        yrr = (1+total_err)**(12/len(_df))-1
        __ls.append(yrr)
    _ls.append(__ls)
    _idx.append(_df.index[0])  
res = pd.DataFrame(_ls, columns = _df.columns, index=_idx)

In [82]:
## res.plot()

In [81]:
## res.plot(kind='box')

In [79]:
dataset_scaled.pct_change().corr()

Ticker,QQQ,SCHD
Ticker,,
QQQ,1.000000,0.749228
SCHD,0.749228,1.000000
